# `AML — Task 2:` Heart rhythm classification from raw ECG signals.

---

**NOTES:** Popular algorithms that can be used for multi-class classification include:
- k-Nearest Neighbors.
- Decision Trees.
- Naive Bayes.
- Random Forest.
- Gradient Boosting.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.metrics import f1_score

## Dataset import and export

In [3]:
def load_from_csv():
    X_train = pd.read_csv('data/X_train.csv').drop(columns=['id'])
    y_train = pd.read_csv('data/y_train.csv').drop(columns=['id'])
    X_test  = pd.read_csv('data/X_test.csv').drop(columns=['id'])
    return X_train, y_train, X_test

In [4]:
def export_to_csv(X_train, y_train, X_test):
    X_train.to_csv('data/X_train_cleaned.csv', index=False)
    y_train.to_csv('data/y_train_cleaned.csv', index=False)
    X_test.to_csv('data/X_test_cleaned.csv', index=False)

## Export `csv` submission file

In [5]:
def create_submission(sub_id, pred, basepath='submissions/task2-sub'):
    result = pred.copy().rename(columns={0: 'y'})
    result['id'] = range(0, len(result))
    result = result[['id', 'y']]
    result.to_csv(basepath + str(sub_id) + '.csv', index=False)

## Data processing `TODO`

In [6]:
X_train, y_train, X_test = load_from_csv()

In [7]:
X_train.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x17832,x17833,x17834,x17835,x17836,x17837,x17838,x17839,x17840,x17841
0,-64,-66,-69,-72,-75,-77,-80,-86,-89,-83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,505,500,496,492,487,480,475,476,483,495,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-21,-16,-12,-7,-3,0,1,2,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-211,-457,-635,-710,-715,-663,-573,-481,-401,-337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36,32,29,25,22,19,17,15,12,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
X_train.describe()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x17832,x17833,x17834,x17835,x17836,x17837,x17838,x17839,x17840,x17841
count,5117.000000,5117.000000,5117.000000,5117.000000,5117.000000,5117.000000,5117.000000,5117.000000,5117.000000,5117.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
mean,6.382842,5.607387,5.115888,4.909713,5.092828,5.723471,6.469220,7.278483,8.176471,9.098886,...,-121.0,-120.0,-118.0,-116.0,-115.0,-114.0,-113.0,-112.0,-111.0,NaN
std,339.371828,337.717355,336.734976,337.096437,338.294857,340.230604,342.761518,345.464582,348.134195,351.312129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,-5888.000000,-5849.000000,-5750.000000,-5638.000000,-5515.000000,-5379.000000,-5233.000000,-5168.000000,-5760.000000,-6279.000000,...,-121.0,-120.0,-118.0,-116.0,-115.0,-114.0,-113.0,-112.0,-111.0,NaN
25%,-89.000000,-89.000000,-89.000000,-89.000000,-87.000000,-87.000000,-85.000000,-85.000000,-86.000000,-86.000000,...,-121.0,-120.0,-118.0,-116.0,-115.0,-114.0,-113.0,-112.0,-111.0,NaN
50%,-9.000000,-10.000000,-11.000000,-10.000000,-10.000000,-10.000000,-9.000000,-8.000000,-8.000000,-8.000000,...,-121.0,-120.0,-118.0,-116.0,-115.0,-114.0,-113.0,-112.0,-111.0,NaN
75%,88.000000,86.000000,86.000000,85.000000,87.000000,87.000000,87.000000,87.000000,87.000000,86.000000,...,-121.0,-120.0,-118.0,-116.0,-115.0,-114.0,-113.0,-112.0,-111.0,NaN
max,5471.000000,5487.000000,5470.000000,5452.000000,5437.000000,5432.000000,5431.000000,5429.000000,5427.000000,5410.000000,...,-121.0,-120.0,-118.0,-116.0,-115.0,-114.0,-113.0,-112.0,-111.0,NaN


## Models defintion `TODO`

## Parameters tuning `TODO`

## Generate new submission

In [9]:
#sub_id = 1
#prediction = pd.DataFrame(model.predict(X_test))

In [10]:
#create_submission(sub_id, prediction)

**Solutions must be submitted on the [project website](https://aml.ise.inf.ethz.ch/task2/).**